<a href="https://colab.research.google.com/github/cornflake15/twitter_sentiment_analysis/blob/master/tweepy-vader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Sentiment Analysis

In [2]:
!pip install vaderSentiment # We only need install vaderSentiment if we run it on Colab

     |████████████████████████████████| 133kB 2.8MB/s 


In [3]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
CONSUMER_API_KEY = '8fxjhgCnLZie0AjbLuntKT0EM'
CONSUMER_API_SECRET = 'd9oab3yrWQybhC64hk5YIXybhuIql20sUWvte2Z8vVQvKtM3Id'
ACCESS_TOKEN ='822833106-VKB22OHa8dXm2Hd5sEl5d5gWLmSfaH5EEmjA88DX'
ACCESS_TOKEN_SECRET = '22iR0cQfOvSQ77UFKKf4CHNLPvvNYEqzNUkugvubrLMx5'
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Utility Function

In [5]:
# Function to get trending topic by country name
def get_trending_topic_by_country(country, num_trends):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    trends_country = api.trends_available()
    for trend in trends_country:
        if trend['name'] == country:
            country_info = (trend['name'], trend['woeid'])
            print('Country: {}, WOEID: {}'.format(trend['name'], trend['woeid']))
    
    index = 1
    trending_topics = []
    trends_result = api.trends_place(country_info[1])
    for tren in trends_result[0]['trends'][:num_trends]: 
        print('#{}: {}'.format(index, tren['name']))
        trending_topics.append(tren['name'])
        index += 1
        
    return trending_topics

# Function to get tweets by keyword
def get_tweets_by_keyword(keywords, num_of_tweets, language, filename='', export_to_csv=False):
    """
    Use this function if you want export the tweet to CSV file
    keywords: tweet keyword
    num_of_tweets: number of tweets that your want to fetch
    language: tweet language
    filename: filename
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
    
    
    if export_to_csv == True:
        export_to_csv(df_tweets, filename)
        
    return df_tweets

# Overiding function 
def get_tweets_by_keyword(keywords, num_of_tweets, language):
    """
    Use this function and its parameter if you don't want export to CSV
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
        
    return df_tweets

# Function to remove unwanted string in the tweet
def remove_unwanted_string(dataframe):
    new_record = []
    tweet_text = dataframe.apply(lambda x: str(x).split())
    for record in tweet_text:
        for text in record:
            if 'RT' in text or '@' in text or '#' in text:
                record.remove(text)
        
        new_record.append(' '.join(record))
    dataframe = new_record
    return dataframe
            
# Function to print sentiments of the sentence
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object
    sid_obj = SentimentIntensityAnalyzer()
    
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary
    # which contains pos, neg, neu, and compoind scores
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict
    
# Function export to CSV
def export_to_csv(dataframe, filename):
    print('\nExported to CSV')
    return dataframe.to_csv(filename)

## Get Tweets by trending topics on spesific country

In [6]:
df_tweets_trending = get_trending_topic_by_country('Indonesia', 10)

Country: Indonesia, WOEID: 23424846
#1: #엑소_3000일_3000만큼_사랑해
#2: #howyoulikethat_d2
#3: #OrangKayaGakTau
#4: #1dmonthbegins
#5: #PBSIHomeTournament
#6: BTS PAVED THE WAY
#7: Pagii
#8: Selamat Hari Bidan Nasional
#9: TREASURE AND TEUMES FOREVER
#10: Kevin


## Get Tweets by Keyword

We can pass trending topic or any keyword as a parameter to the function.

In [7]:
df_tweets = get_tweets_by_keyword(['covid', 'covid-19'], 100, 'en')

In [8]:
df_tweets

,username,screen_name,user_location,user_verified,followers_count,friends_count,listed_count,tweet_created_at,tweet_text,tweet_coordinates,tweet_place,tweet_favorite_count
0,jaykeep,jaykeeper10,,False,340,213,0,2020-06-24 03:27:21,RT @Carlos4PF: My Uncle Gustavo opened a chick...,None,None,0
1,Adarsh Tiwari,aadarshtiwary,"Lucknow, India",False,7,19,0,2020-06-24 03:27:21,RT @TOILucknow: Lucknow: The Lucknow Universit...,None,None,0
2,G. DeYoung,vikingsarehere,"Los Angeles, CA",False,1679,2373,1,2020-06-24 03:27:21,RT @Jar_O_Cats: Should be required reading for...,None,None,0
3,Mela Storm #Resistance! NO SURRENDER,StormMela,"Kansas, USA",False,27794,18678,10,2020-06-24 03:27:21,RT @LauraKellyKS: If we don't improve KS' #202...,None,None,0
4,Ray King,king_of_ray,,False,22,11,0,2020-06-24 03:27:21,@morethanmySLE Trump is more than a laughing s...,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Hiro Okabe,mcx5ho1,日本,False,59,237,1,2020-06-24 03:27:11,Business has a unique role to play today. Not ...,None,None,0
96,Ashwini Hiremath 🌎,TheWiseScalpel,,False,224,248,0,2020-06-24 03:27:11,RT @ndtv: Delhi's Class 10 student makes face ...,None,None,0
97,Cyclops OmniView,CobusDuPlessis6,"Johannesburg, South Africa",False,2,44,0,2020-06-24 03:27:10,Covid-19 spread dynamics:\nMythbusters Season ...,None,None,0
98,DeAndre Thompson,deandreignt,,False,746,838,9,2020-06-24 03:27:10,RT @Acyn: The President uses the term “Kung Fl...,None,None,0


In [9]:
df_tweets['tweet_text'][1]

'RT @TOILucknow: Lucknow: The Lucknow University Associated College Teachers’ Association (LUACTA) has threatened to boycott undergraduate a…'

## Remove Unwanted String from Tweets

In [10]:
df_tweets['tweet_text'] = remove_unwanted_string(df_tweets['tweet_text'])
print(df_tweets['tweet_text'][1])

@TOILucknow: Lucknow: The Lucknow University Associated College Teachers’ Association (LUACTA) has threatened to boycott undergraduate a…


In [11]:
# export_to_csv(df_tweets['tweet_text'], '../dataset/tweet_sample_100.csv')

## Tweet Sentiment with VaderSentiment library/module

Note: Can not support Indonesian language

In [12]:
pos, neg, neu, comp = [], [], [], []
for text in df_tweets['tweet_text']:
    sentiment = sentiment_scores(text)
    pos.append(sentiment['pos'])
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    comp.append(sentiment['compound'])
    
df_tweets_sentiment = pd.DataFrame()
# df_tweets_sentiment.columns = ['tweet_text', 'positive', 'negative', 'neutral', 'compound']
df_tweets_sentiment['tweet_text'] = df_tweets['tweet_text']
df_tweets_sentiment['positive'] = pos
df_tweets_sentiment['negative'] = neg
df_tweets_sentiment['neutral'] = neu
df_tweets_sentiment['compound'] = comp

In [13]:
df_tweets_sentiment

,tweet_text,positive,negative,neutral,compound
0,@Carlos4PF: My Uncle Gustavo opened a chicken ...,0.144,0.097,0.758,0.0772
1,@TOILucknow: Lucknow: The Lucknow University A...,0.000,0.275,0.725,-0.6486
2,@Jar_O_Cats: Should be required reading for ev...,0.115,0.250,0.635,-0.5256
3,@LauraKellyKS: If we don't improve KS' partici...,0.000,0.447,0.553,-0.8274
4,"Trump is more than a laughing stock, he is a p...",0.134,0.128,0.737,0.0422
...,...,...,...,...,...
95,Business has a unique role to play today. Not ...,0.082,0.000,0.918,0.1779
96,@ndtv: Delhi's Class 10 student makes face shi...,0.000,0.148,0.852,-0.3818
97,Covid-19 spread dynamics: Mythbusters Season 8...,0.085,0.230,0.685,-0.5574
98,@Acyn: The President uses the term “Kung Flu” ...,0.000,0.000,1.000,0.0000


Detail information about the output visit https://github.com/cjhutto/vaderSentiment